In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="18Lc8XC_lV-uzRcNcgg-LLZXfkKSQ0vDX", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

# Masked Diffusion from First Principles

*Part 1 of the Vizuara series on Diffusion Language Models*
*Estimated time: 45 minutes*

## 1. Why Does This Matter?

Every large language model you have used — GPT-4, Claude, LLaMA — generates text **one token at a time**, left to right. This is called **autoregressive** generation, and it works like a typewriter: once a word is committed, there is no going back.

But what if a language model could work like a **painter** instead? Start with a blank canvas, sketch a rough outline, and then progressively refine every part simultaneously?

This is exactly what **Diffusion Language Models** do. Instead of generating text left-to-right, they start with a fully masked sequence and iteratively reveal tokens — unmasking the ones they are most confident about first.

By the end of this notebook, you will have built a complete masked diffusion language model from scratch and watched it generate text through iterative unmasking. Here is a preview of what the generation process looks like:

```
Step 1: [M] [M] [M] [M] [M] [M] [M] [M]
Step 2: [M] [M]  c  [M] [M]  a  [M] [M]
Step 3:  a  [M]  c  [M]  b   a  [M]  c
Step 4:  a   b   c   a   b   a   b   c
```

The model fills in the easiest tokens first, then uses that context to resolve the harder ones. Let us build this from scratch.

In [ ]:
# 🔧 Setup — Run this cell first
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math
from IPython.display import clear_output

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

### The Core Problem: How Do You "Noise" Text?

In image diffusion, the forward process gradually adds Gaussian noise to pixels. A pixel value of 127.0 becomes 128.1 — still a valid pixel.

But text is **discrete**. The word "cat" is a symbol, not a number. You cannot add "a little bit of noise" to "cat" and get something between "cat" and "dog." It simply does not make sense.

The solution? **Masking.** Instead of adding noise, we replace tokens with a special `[MASK]` token. This is the text equivalent of corrupting an image with noise:

| Image Diffusion | Text Diffusion |
|---|---|
| Add Gaussian noise | Replace tokens with [MASK] |
| Pure noise at t=1 | All [MASK] at t=1 |
| Predict noise to remove | Predict original token at [MASK] |
| Continuous | Discrete |

### 🤔 Think About This

Before we write any code, consider this question: **In what order should a diffusion language model reveal tokens during generation?**

An autoregressive model has no choice — it must go left to right. But a diffusion model can unmask tokens in *any* order. Should it unmask left to right? Random order? Or something smarter?

*Think about this for a moment. We will see the answer emerge naturally from the math.*

In [ ]:
#@title 🎧 Listen: Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics

### The Forward Process

Given a clean sequence $x_0 = [x_0^1, x_0^2, \ldots, x_0^L]$ of $L$ tokens, the forward process at time $t \in [0, 1]$ independently masks each token with probability $t$:

$$q(x_t^i \mid x_0^i) = (1 - t) \cdot \mathbb{1}[x_t^i = x_0^i] + t \cdot \mathbb{1}[x_t^i = \texttt{[MASK]}]$$

**What this says computationally:** For each token, flip a biased coin with probability $t$. Heads → replace with [MASK]. Tails → keep the original.

At $t = 0$, nothing is masked (clean text). At $t = 1$, everything is masked (pure noise). At $t = 0.5$, about half the tokens are masked.

### The Training Objective

The loss function asks the model to predict the original tokens at masked positions:

$$\mathcal{L} = -\mathbb{E}_{t \sim U(0,1)} \left[ \frac{1}{t \cdot L} \sum_{i:\, x_t^i = \texttt{[MASK]}} \log p_\theta(x_0^i \mid x_t) \right]$$

**What this says computationally:**
1. Pick a random masking ratio $t$ between 0 and 1
2. Mask that fraction of tokens
3. Feed the partially masked sequence into the Transformer
4. Compute cross-entropy loss at masked positions only
5. Divide by $t \cdot L$ (the expected number of masks) to normalize

The $1/t$ weighting is crucial — it upweights low masking ratios (where few tokens are masked but each prediction is harder because there is less context).

**Numerical example:** Suppose $L = 4$, $t = 0.5$, and 2 tokens are masked. The model predicts the correct token at position 2 with probability 0.8 and at position 3 with probability 0.6:

$$\mathcal{L} = -\frac{1}{0.5 \times 4}[\log(0.8) + \log(0.6)] = -\frac{1}{2}[-0.223 + (-0.511)] = 0.367$$

The loss is 0.367. If the model had predicted both tokens perfectly (probability 1.0), the loss would be 0. Training pushes the model toward more confident, accurate predictions.

In [ ]:
#@title 🎧 Listen: Data Masking
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_data_masking.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let's Build It — Component by Component

### 4.1 The Synthetic Dataset

We will train on a simple synthetic dataset: sequences built from repeating patterns. This is small enough to train in minutes but complex enough to test whether our diffusion model truly learns structure.

In [ ]:
# --- Configuration ---
VOCAB_SIZE = 8       # Tokens: [MASK]=0, then 1-7 are our vocabulary
SEQ_LEN = 16         # Sequence length
MASK_TOKEN = 0       # Token ID for [MASK]
BATCH_SIZE = 64
D_MODEL = 64         # Embedding dimension
N_HEADS = 4          # Attention heads
N_LAYERS = 3         # Transformer layers

# --- Synthetic Data Generator ---
def generate_pattern_data(batch_size, seq_len, vocab_size):
    """Generate sequences with learnable repeating patterns.

    Each sequence picks a random short pattern (length 2-4) from the
    vocabulary and tiles it to fill the sequence length. The model
    must learn to recognize and complete these patterns.
    """
    sequences = []
    for _ in range(batch_size):
        pattern_len = np.random.randint(2, 5)  # pattern of length 2-4
        # Tokens 1 to vocab_size-1 (avoid 0 which is MASK)
        pattern = np.random.randint(1, vocab_size, size=pattern_len)
        # Tile to fill sequence
        seq = np.tile(pattern, seq_len // pattern_len + 1)[:seq_len]
        sequences.append(seq)
    return torch.tensor(np.array(sequences), dtype=torch.long).to(device)

# Let's see some example patterns
examples = generate_pattern_data(5, SEQ_LEN, VOCAB_SIZE)
for i, seq in enumerate(examples):
    tokens = seq.tolist()
    print(f"Pattern {i+1}: {tokens}")

### 4.2 The Forward Masking Process

This is the heart of the diffusion forward process — randomly masking tokens with probability $t$.

In [ ]:
def mask_tokens(x_0, t):
    """Apply the forward masking process.

    Args:
        x_0: Clean token sequences, shape (B, L)
        t: Masking ratio for each sample, shape (B, 1)

    Returns:
        x_t: Masked sequences, shape (B, L)
        mask: Boolean mask indicating which positions were masked, shape (B, L)
    """
    # For each token, independently mask with probability t
    random_vals = torch.rand_like(x_0.float())           # (B, L)
    mask = random_vals < t                                # (B, L) — True where masked
    x_t = x_0.clone()
    x_t[mask] = MASK_TOKEN
    return x_t, mask

In [ ]:
# 📊 Visualization: The forward process at different timesteps
fig, axes = plt.subplots(1, 5, figsize=(18, 3))
sample = generate_pattern_data(1, SEQ_LEN, VOCAB_SIZE)

timesteps = [0.0, 0.25, 0.5, 0.75, 1.0]
for ax, t_val in zip(axes, timesteps):
    t = torch.tensor([[t_val]]).to(device)
    masked, _ = mask_tokens(sample, t)

    # Visualize as colored grid
    colors = plt.cm.Set2(sample[0].cpu().numpy() / VOCAB_SIZE)
    display = masked[0].cpu().numpy()

    for pos in range(SEQ_LEN):
        if display[pos] == MASK_TOKEN:
            ax.add_patch(plt.Rectangle((pos, 0), 1, 1, color='black', alpha=0.8))
            ax.text(pos + 0.5, 0.5, 'M', ha='center', va='center',
                    color='white', fontsize=9, fontweight='bold')
        else:
            ax.add_patch(plt.Rectangle((pos, 0), 1, 1,
                         color=plt.cm.Set2(display[pos] / VOCAB_SIZE)))
            ax.text(pos + 0.5, 0.5, str(display[pos]), ha='center',
                    va='center', fontsize=9)

    ax.set_xlim(0, SEQ_LEN)
    ax.set_ylim(0, 1)
    ax.set_title(f't = {t_val}', fontsize=13)
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('Forward Process: Gradually Masking Tokens', fontsize=15, y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Transformer
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_transformer.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 The Bidirectional Transformer

Our model is a standard Transformer encoder — crucially, with **no causal mask**. This means every position can attend to every other position, including both masked and unmasked tokens. We also add a **time embedding** so the model knows the current masking ratio.

In [ ]:
class PositionalEncoding(nn.Module):
    """Standard sinusoidal positional encoding."""
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
class DiffusionLM(nn.Module):
    """A bidirectional Transformer for masked diffusion language modeling."""

    def __init__(self, vocab_size, d_model, n_heads, n_layers, max_len=512):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len)
        self.time_mlp = nn.Sequential(
            nn.Linear(1, d_model),
            nn.SiLU(),
            nn.Linear(d_model, d_model),
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=0.1,
            batch_first=True,
            norm_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, n_layers)
        self.output_head = nn.Linear(d_model, vocab_size)

    def forward(self, x_t, t):
        """
        Args:
            x_t: Masked token IDs, shape (B, L)
            t: Masking ratio, shape (B, 1)
        Returns:
            Logits over vocabulary at every position, shape (B, L, V)
        """
        # Token embeddings + positional encoding
        h = self.token_embed(x_t)           # (B, L, D)
        h = self.pos_enc(h)

        # Add time conditioning (broadcast to all positions)
        t_emb = self.time_mlp(t).unsqueeze(1)  # (B, 1, D)
        h = h + t_emb

        # Bidirectional Transformer — NO causal mask!
        h = self.transformer(h)

        # Project to vocabulary
        return self.output_head(h)           # (B, L, V)


model = DiffusionLM(VOCAB_SIZE, D_MODEL, N_HEADS, N_LAYERS).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,}")

### 💡 Key Insight: Why No Causal Mask?

In GPT-style models, the attention mask prevents each position from seeing future tokens (causal masking). This enforces left-to-right generation.

In our diffusion model, we **want** every position to see every other position — both masked and unmasked. This bidirectional attention is what allows the model to:
- Use unmasked tokens on the *right* to predict masked tokens on the *left*
- Fill in tokens in any order, not just left-to-right
- Overcome the "reversal curse" that plagues autoregressive models

In [ ]:
#@title 🎧 Listen: Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 The Training Loop

In [ ]:
def train_diffusion_lm(model, n_steps=2000, lr=3e-4):
    """Train the masked diffusion language model."""
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_steps)
    losses = []

    for step in range(n_steps):
        # Generate a fresh batch of pattern data
        x_0 = generate_pattern_data(BATCH_SIZE, SEQ_LEN, VOCAB_SIZE)

        # Sample random masking ratio t ~ U(epsilon, 1) for each sample
        # We use epsilon > 0 to avoid division by zero in the weighting
        t = torch.rand(BATCH_SIZE, 1, device=device) * 0.98 + 0.02  # (B, 1)

        # Apply forward masking process
        x_t, mask = mask_tokens(x_0, t)

        # Forward pass: predict original tokens
        logits = model(x_t, t)                # (B, L, V)

        # Compute loss ONLY at masked positions
        # Flatten for cross-entropy
        logits_masked = logits[mask]           # (N_masked, V)
        targets_masked = x_0[mask]             # (N_masked,)

        if logits_masked.shape[0] == 0:
            continue  # Skip if nothing was masked

        loss = F.cross_entropy(logits_masked, targets_masked)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        if (step + 1) % 200 == 0:
            print(f"Step {step+1}/{n_steps} | Loss: {loss.item():.4f}")

    return losses

print("Training the diffusion language model...")
losses = train_diffusion_lm(model, n_steps=2000)
print("Done!")

In [ ]:
# 📊 Training curve
plt.figure(figsize=(10, 4))
# Smooth with moving average
window = 50
smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
plt.plot(smoothed, color='#1565c0', linewidth=2)
plt.xlabel('Training Step', fontsize=12)
plt.ylabel('Cross-Entropy Loss', fontsize=12)
plt.title('Diffusion LM Training Loss', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Todo
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_todo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5. 🔧 Your Turn: Implement the Generation Algorithm

Now comes the exciting part — generation! The model starts with all [MASK] tokens and iteratively unmasks them.

The key idea: at each step, predict all masked tokens, but only **keep the most confident predictions**. Remask the uncertain ones for the next step.

### TODO: Complete the `generate` function

In [ ]:
@torch.no_grad()
def generate(model, seq_len=SEQ_LEN, n_steps=8):
    """Generate a sequence using iterative confidence-based unmasking.

    Args:
        model: Trained DiffusionLM
        seq_len: Length of sequence to generate
        n_steps: Number of denoising steps

    Returns:
        final_sequence: Generated token IDs, shape (1, seq_len)
        history: List of (sequence, step) tuples for visualization
    """
    model.eval()
    # Start fully masked
    x = torch.full((1, seq_len), MASK_TOKEN, dtype=torch.long, device=device)
    history = [(x[0].cpu().clone(), 'Start')]

    for s in range(n_steps, 0, -1):
        t = torch.tensor([[s / n_steps]], device=device, dtype=torch.float)
        logits = model(x, t)
        probs = F.softmax(logits, dim=-1)  # (1, L, V)

        # ============ TODO ============
        # Step 1: Sample a token from the predicted distribution at every position
        #         Use torch.multinomial on the flattened probs
        #         Hint: probs.view(-1, VOCAB_SIZE) gives shape (L, V)
        sampled = ???  # YOUR CODE: shape should be (1, seq_len)

        # Step 2: Compute confidence = probability of the sampled token
        #         Use probs.gather(-1, ...) to look up the probability of each sampled token
        confidence = ???  # YOUR CODE: shape should be (1, seq_len)

        # Step 3: Only consider masked positions
        #         Set confidence of already-unmasked positions to infinity
        #         so they are never selected for unmasking (they're already done)
        is_masked = (x == MASK_TOKEN)
        confidence[~is_masked] = ???  # YOUR CODE

        # Step 4: Determine how many tokens to unmask this step
        #         We want to unmask roughly (1/s) of the remaining masked tokens
        n_to_unmask = max(1, int(is_masked.sum().item() * (1.0 / s)))

        # Step 5: Find the n_to_unmask positions with HIGHEST confidence among masked ones
        #         But we set non-masked to inf, so we need the LOWEST confidence values
        #         actually — we want highest confidence among masked tokens
        #         Re-think: set non-masked to -inf so topk picks the masked ones with highest conf
        # ==============================

        # Fix: set non-masked to -inf for topk selection
        conf_for_selection = confidence.clone()
        conf_for_selection[~is_masked] = -float('inf')

        _, top_idx = conf_for_selection.topk(n_to_unmask, dim=-1)
        x.scatter_(1, top_idx, sampled.gather(1, top_idx))

        history.append((x[0].cpu().clone(), f'Step {n_steps - s + 1}'))

    return x, history

In [ ]:
# ✅ Verification: Test your implementation
try:
    generated, history = generate(model, seq_len=SEQ_LEN, n_steps=8)
    assert generated.shape == (1, SEQ_LEN), f"Wrong shape: {generated.shape}"
    assert (generated != MASK_TOKEN).all(), "Some positions still masked!"
    print("✅ Generation works! Here's the output:")
    print(f"   Generated sequence: {generated[0].tolist()}")
    print(f"   Steps recorded: {len(history)}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Check your TODO implementation above.")

In [ ]:
#@title 🎧 Listen: Post Todo
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_post_todo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

---
### ✋ Stop and Think
Before running the solution, try to answer:
1. Why do we set confidence of already-unmasked positions to negative infinity?
2. What happens if we unmask ALL tokens in one step instead of gradually?
3. Why do function words (common tokens) tend to appear first?

*Take a minute. Then scroll down for the solution.*

---

### Solution

In [ ]:
@torch.no_grad()
def generate(model, seq_len=SEQ_LEN, n_steps=8):
    """Generate a sequence using iterative confidence-based unmasking."""
    model.eval()
    x = torch.full((1, seq_len), MASK_TOKEN, dtype=torch.long, device=device)
    history = [(x[0].cpu().clone(), 'Start')]

    for s in range(n_steps, 0, -1):
        t = torch.tensor([[s / n_steps]], device=device, dtype=torch.float)
        logits = model(x, t)
        probs = F.softmax(logits, dim=-1)

        # Step 1: Sample tokens from predicted distributions
        sampled = torch.multinomial(
            probs.view(-1, VOCAB_SIZE), num_samples=1
        ).view(1, -1)  # (1, L)

        # Step 2: Confidence = probability of the sampled token
        confidence = probs.gather(-1, sampled.unsqueeze(-1)).squeeze(-1)  # (1, L)

        # Step 3: Only unmask among currently masked positions
        is_masked = (x == MASK_TOKEN)

        # Step 4: How many to unmask this step
        n_to_unmask = max(1, int(is_masked.sum().item() * (1.0 / s)))

        # Step 5: Pick the most confident masked positions
        conf_for_selection = confidence.clone()
        conf_for_selection[~is_masked] = -float('inf')

        _, top_idx = conf_for_selection.topk(n_to_unmask, dim=-1)
        x.scatter_(1, top_idx, sampled.gather(1, top_idx))

        history.append((x[0].cpu().clone(), f'Step {n_steps - s + 1}'))

    return x, history

# Generate and show
generated, history = generate(model, seq_len=SEQ_LEN, n_steps=8)
print("Generated:", generated[0].tolist())

In [ ]:
#@title 🎧 Listen: Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 6. Putting It All Together

In [ ]:
# 📊 Visualize the full generation process step by step
def visualize_generation(history, vocab_size):
    """Show how tokens are revealed step by step."""
    n_steps = len(history)
    fig, axes = plt.subplots(n_steps, 1, figsize=(14, n_steps * 0.8))

    if n_steps == 1:
        axes = [axes]

    for ax, (seq, label) in zip(axes, history):
        tokens = seq.numpy()
        for pos in range(len(tokens)):
            if tokens[pos] == MASK_TOKEN:
                ax.add_patch(plt.Rectangle((pos, 0), 1, 1,
                             color='#333333', alpha=0.85))
                ax.text(pos + 0.5, 0.5, 'M', ha='center', va='center',
                        color='white', fontsize=10, fontweight='bold')
            else:
                color = plt.cm.Set2(tokens[pos] / vocab_size)
                ax.add_patch(plt.Rectangle((pos, 0), 1, 1, color=color))
                ax.text(pos + 0.5, 0.5, str(tokens[pos]), ha='center',
                        va='center', fontsize=10)

        ax.set_xlim(0, len(tokens))
        ax.set_ylim(0, 1)
        ax.set_ylabel(label, fontsize=10, rotation=0, ha='right', va='center')
        ax.set_xticks([])
        ax.set_yticks([])

    plt.suptitle('Generation Process: Iterative Unmasking',
                 fontsize=15, y=1.02)
    plt.tight_layout()
    plt.show()


generated, history = generate(model, seq_len=SEQ_LEN, n_steps=8)
visualize_generation(history, VOCAB_SIZE)

## 7. Evaluating the Model

Let us check whether the model actually learned the repeating patterns. We can test this by masking part of a known pattern and seeing if the model recovers it.

In [ ]:
def evaluate_pattern_completion(model, n_tests=100):
    """Test if the model can complete partially masked patterns."""
    correct = 0
    total = 0

    for _ in range(n_tests):
        # Generate a clean pattern
        x_0 = generate_pattern_data(1, SEQ_LEN, VOCAB_SIZE)

        # Mask the last half
        x_t = x_0.clone()
        x_t[0, SEQ_LEN // 2:] = MASK_TOKEN
        t = torch.tensor([[0.5]], device=device)

        # Predict
        logits = model(x_t, t)
        preds = logits[0, SEQ_LEN // 2:].argmax(dim=-1)
        targets = x_0[0, SEQ_LEN // 2:]

        correct += (preds == targets).sum().item()
        total += len(targets)

    accuracy = correct / total * 100
    return accuracy

accuracy = evaluate_pattern_completion(model)
print(f"Pattern completion accuracy: {accuracy:.1f}%")

In [ ]:
# 📊 Generate multiple sequences and display them
print("Generated sequences (should show repeating patterns):\n")
for i in range(8):
    generated, _ = generate(model, seq_len=SEQ_LEN, n_steps=10)
    seq = generated[0].tolist()
    print(f"  Sequence {i+1}: {seq}")

In [ ]:
#@title 🎧 Listen: Final
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_final.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 8. 🎯 Final Output: Animated Generation

In [ ]:
def animated_generation_grid(model, n_sequences=6, n_steps=10):
    """Generate multiple sequences and show the unmasking process as a grid."""
    fig, axes = plt.subplots(n_sequences, n_steps + 1, figsize=(18, n_sequences * 1.2))

    for row in range(n_sequences):
        _, history = generate(model, seq_len=SEQ_LEN, n_steps=n_steps)

        # Pad history if needed
        while len(history) < n_steps + 1:
            history.append(history[-1])

        for col in range(n_steps + 1):
            ax = axes[row, col]
            seq = history[col][0].numpy()

            # Create colored visualization
            img = np.zeros((1, SEQ_LEN, 3))
            for pos in range(SEQ_LEN):
                if seq[pos] == MASK_TOKEN:
                    img[0, pos] = [0.2, 0.2, 0.2]  # dark gray for MASK
                else:
                    c = plt.cm.Set2(seq[pos] / VOCAB_SIZE)[:3]
                    img[0, pos] = c

            ax.imshow(img, aspect='auto', interpolation='nearest')
            ax.set_xticks([])
            ax.set_yticks([])

            if row == 0:
                ax.set_title(history[col][1], fontsize=9)
            if col == 0:
                ax.set_ylabel(f'Seq {row+1}', fontsize=9)

    plt.suptitle('Diffusion LM Generation: From Masked to Revealed',
                 fontsize=15, y=1.02)
    plt.tight_layout()
    plt.show()
    print("🎉 Each row shows one sequence being generated through iterative unmasking!")
    print("   Dark = [MASK], Colors = revealed tokens. Notice how easy tokens appear first.")

animated_generation_grid(model)

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 9. Reflection and Next Steps

### 🤔 Reflection Questions

1. **Order of unmasking:** Did you notice which tokens tend to be revealed first? Why might common tokens (those that appear frequently in patterns) be unmasked earlier?

2. **Number of steps:** What happens if you set `n_steps=1` (unmask everything in one shot)? Why is the quality worse? What about `n_steps=50`?

3. **Comparison to autoregressive:** If we had built an autoregressive model on the same data, it would generate left-to-right. What advantage does our diffusion model have for pattern completion?

### 🏆 Optional Challenges

1. **Temperature sampling:** Add a `temperature` parameter to the generation function. How does temperature affect the diversity vs quality tradeoff?

2. **Different data:** Swap the synthetic patterns for a character-level text dataset. Does the model learn to generate readable text?

3. **Masking schedules:** Instead of uniform random $t$, try a cosine schedule where more training time is spent at high masking ratios. Does this improve generation quality?

**Next notebook:** We will demonstrate the **reversal curse** — training both an autoregressive model and a diffusion model on the same data, and showing that only the diffusion model can reason bidirectionally.

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))